In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls

gdrive	sample_data


In [0]:
## import necessary libraries
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder ## For label encoding(converting categorical values to label)
from xgboost import XGBClassifier ## XG boost model
from sklearn.model_selection import GridSearchCV ##For Grid search(cross validation)
from sklearn.metrics import accuracy_score ## For getting accuracy value
from sklearn.metrics import confusion_matrix,classification_report ## For classifier metrics(accuracy,TPR,TNR)
from sklearn.ensemble import GradientBoostingClassifier ## GradientBoost Model

In [0]:
train_data=pd.read_csv('/content/gdrive/My Drive/Analytic_vidya/train_fNxu4vz.csv')
test_data=pd.read_csv('/content/gdrive/My Drive/Analytic_vidya/test_fjtUOL8.csv')

In [343]:
print(train_data.shape)
print(test_data.shape)

(164309, 14)
(109541, 13)


In [344]:
train_data.head(3)

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3


In [345]:
test_data.head(3)

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
0,10164310,"27,500",10+ years,Mortgage,129000.0,VERIFIED - income,debt_consolidation,12.87,0,68.0,10,37,Male
1,10164311,"26,000",10+ years,NaN,110000.0,not verified,credit_card,11.37,0,NaN,6,23,Male
2,10164312,"6,075",< 1 year,Rent,75000.0,VERIFIED - income,debt_consolidation,6.83,2,NaN,5,20,Male


In [346]:
train_data.tail(3)

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
164306,10164307,"20,250",10+ years,Mortgage,NaN,VERIFIED - income,credit_card,18.17,1,NaN,17,30,Male,1
164307,10164308,"10,000",4 years,Rent,NaN,not verified,major_purchase,18.75,1,48.0,16,62,Female,3
164308,10164309,"9,250",10+ years,Rent,NaN,VERIFIED - income,credit_card,19.44,1,NaN,5,9,Female,2


In [347]:
test_data.tail(3)

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
109538,10273848,"2,000",5 years,Rent,37850.0,not verified,credit_card,11.60,0,NaN,10,11,Male
109539,10273849,"2,125",6 years,Rent,30000.0,not verified,other,24.88,0,NaN,5,6,Male
109540,10273850,"15,000",2 years,Mortgage,137000.0,not verified,medical,8.66,1,60.0,8,17,Male


### Understanding the data

In [0]:
def understand_data(data) :
    return(pd.DataFrame({"Datatype":data.dtypes,"No of NAs":data.isna().sum(),
                         "No of Levels":data.apply(lambda x: x.nunique(),axis=0),"Levels":data.apply(lambda x: str(x.unique()),axis=0)}))

In [349]:
understand_data(train_data)

,Datatype,No of NAs,No of Levels,Levels
Loan_ID,int64,0,164309,[10000001 10000002 10000003 ... 10164307 10164...
Loan_Amount_Requested,object,0,1290,"['7,000' '30,000' '24,725' ... '28,950' '33,32..."
Length_Employed,object,7371,11,['< 1 year' '4 years' '7 years' '8 years' '2 y...
Home_Owner,object,25349,5,['Rent' 'Mortgage' nan 'Own' 'Other' 'None']
Annual_Income,float64,25102,12305,[68000.0 nan 75566.4 ... 51024.0 28721.4 24864.0]
Income_Verified,object,0,3,['not verified' 'VERIFIED - income' 'VERIFIED ...
Purpose_Of_Loan,object,0,14,['car' 'debt_consolidation' 'credit_card' 'hom...
Debt_To_Income,float64,0,3953,[18.37 14.93 15.88 ... 38.03 39.04 35.06]
Inquiries_Last_6Mo,int64,0,9,[0 3 1 2 4 5 6 7 8]
Months_Since_Deliquency,float64,88379,122,[nan 17.0 16.0 68.0 13.0 6.0 64.0 10.0 63.0 33...


In [350]:
understand_data(test_data)

,Datatype,No of NAs,No of Levels,Levels
Loan_ID,int64,0,109541,[10164310 10164311 10164312 ... 10273848 10273...
Loan_Amount_Requested,object,0,1246,"['27,500' '26,000' '6,075' ... '24,050' '34,42..."
Length_Employed,object,4936,11,['10+ years' '< 1 year' '6 years' '8 years' '1...
Home_Owner,object,16711,5,['Mortgage' nan 'Rent' 'Own' 'Other' 'None']
Annual_Income,float64,16898,9028,[129000.0 110000.0 75000.0 ... 204600.0 49680....
Income_Verified,object,0,3,['VERIFIED - income' 'not verified' 'VERIFIED ...
Purpose_Of_Loan,object,0,14,['debt_consolidation' 'credit_card' 'home_impr...
Debt_To_Income,float64,0,3895,[12.87 11.37 6.83 ... 35.86 38.78 38.76]
Inquiries_Last_6Mo,int64,0,9,[0 2 1 3 6 4 5 7 8]
Months_Since_Deliquency,float64,58859,115,[68.0 nan 26.0 18.0 22.0 65.0 47.0 45.0 49.0 6...


In [0]:
train_data['Loan_Amount_Requested'] = train_data['Loan_Amount_Requested'].str.replace(',','')


In [0]:
test_data['Loan_Amount_Requested'] = test_data['Loan_Amount_Requested'].str.replace(',','')

In [354]:
print(train_data['Loan_Amount_Requested'].head())
print(test_data['Loan_Amount_Requested'].head())

0     7000
1    30000
2    24725
3    16000
4    17000
Name: Loan_Amount_Requested, dtype: object
0    27500
1    26000
2     6075
3    12000
4    35000
Name: Loan_Amount_Requested, dtype: object


In [0]:
train_data['Loan_Amount_Requested'] = train_data['Loan_Amount_Requested'].astype(float)
test_data['Loan_Amount_Requested'] = test_data['Loan_Amount_Requested'].astype(float)

In [0]:
### data type conversion for train data
cols=['Length_Employed','Home_Owner','Income_Verified','Purpose_Of_Loan','Gender']
train_data[cols] = train_data[cols].astype('category')

In [0]:
### data type conversion for test data
test_data[cols] = test_data[cols].astype('category')

In [359]:
train_data.dtypes

Loan_ID                       int64
Loan_Amount_Requested       float64
Length_Employed            category
Home_Owner                 category
Annual_Income               float64
Income_Verified            category
Purpose_Of_Loan            category
Debt_To_Income              float64
Inquiries_Last_6Mo            int64
Months_Since_Deliquency     float64
Number_Open_Accounts          int64
Total_Accounts                int64
Gender                     category
Interest_Rate                 int64
dtype: object

In [360]:
test_data.dtypes

Loan_ID                       int64
Loan_Amount_Requested       float64
Length_Employed            category
Home_Owner                 category
Annual_Income               float64
Income_Verified            category
Purpose_Of_Loan            category
Debt_To_Income              float64
Inquiries_Last_6Mo            int64
Months_Since_Deliquency     float64
Number_Open_Accounts          int64
Total_Accounts                int64
Gender                     category
dtype: object

In [0]:
## set index for train and test
train_data.set_index('Loan_ID',inplace=True)
test_data.set_index('Loan_ID',inplace=True)

In [0]:
### Filling the missing values
### Before that Splitting the train into train and validation data
X_train,X_validation,y_train,y_validation = train_test_split(train_data.drop('Interest_Rate',axis=1),train_data['Interest_Rate'],
                                                             test_size=0.2,random_state=1234)

In [364]:
print(X_train.shape)
print(y_train.shape)
print(X_validation.shape)
print(y_validation.shape)

(131447, 12)
(131447,)
(32862, 12)
(32862,)


In [365]:
X_train['Length_Employed'].value_counts()

10+ years    42290
2 years      11673
3 years      10419
< 1 year     10202
5 years       8678
1 year        8409
4 years       7789
7 years       7437
6 years       7256
8 years       6356
9 years       5059
Name: Length_Employed, dtype: int64

In [0]:
### Length_Employed has values which has to be in ordinal form for which mapping is done
experience_map={
    '< 1 year':'0',
    '1 year':'1',
    '2 years':'2',
    '3 years':'3',
    '4 years':'4',
    '5 years':'5',
    '6 years':'6',
    '7 years':'7',
    '8 years':'8',
    '9 years':'9',
    '10+ years': '10',
    'NaN':'Unknown',
}

In [0]:
def experience_emp(exp):
    return experience_map[exp]

In [0]:
X_train['Length_Employed']=X_train['Length_Employed'].apply(experience_emp)

In [369]:
X_train['Length_Employed'].head(3)

Loan_ID
10003909    10
10120777     2
10129626     2
Name: Length_Employed, dtype: category
Categories (11, object): [1, 10, 2, 3, ..., 7, 8, 9, 0]

In [0]:
X_validation['Length_Employed']=X_validation['Length_Employed'].apply(experience_emp)

In [371]:
X_validation['Length_Employed'].head(3)

Loan_ID
10086865    7
10123833    9
10066639    8
Name: Length_Employed, dtype: category
Categories (11, object): [1, 10, 2, 3, ..., 7, 8, 9, 0]

In [0]:
test_data['Length_Employed']=test_data['Length_Employed'].apply(experience_emp)

In [373]:
test_data['Length_Employed'].head(3)

Loan_ID
10164310    10
10164311    10
10164312     0
Name: Length_Employed, dtype: category
Categories (11, object): [1, 10, 2, 3, ..., 7, 8, 9, 0]

In [0]:
## Filling NA values requires a value that already exists as a category for categgorical columns
X_train['Length_Employed'] = X_train['Length_Employed'].cat.add_categories('Unknown')
X_train['Length_Employed'].fillna('Unknown',inplace=True)

In [0]:
X_validation['Length_Employed'] = X_validation['Length_Employed'].cat.add_categories('Unknown')
X_validation['Length_Employed'].fillna('Unknown',inplace=True)

In [0]:
X_train['Home_Owner'].fillna('None',inplace=True) ### Filling empty values with None Category
X_validation['Home_Owner'].fillna('None',inplace=True)

In [0]:
test_data['Length_Employed'] = test_data['Length_Employed'].cat.add_categories('Unknown')
test_data['Length_Employed'].fillna('Unknown',inplace=True)

In [0]:
test_data['Home_Owner'].fillna('None',inplace=True)

In [0]:
X_train['Annual_Income'].fillna(X_train['Annual_Income'].mean(),inplace=True)
X_validation['Annual_Income'].fillna(X_validation['Annual_Income'].mean(),inplace=True)

In [0]:
test_data['Annual_Income'].fillna(test_data['Annual_Income'].mean(),inplace=True)

In [0]:
X_train['Months_Since_Deliquency'].fillna(X_train['Months_Since_Deliquency'].mean(),inplace=True)
X_validation['Months_Since_Deliquency'].fillna(X_validation['Months_Since_Deliquency'].mean(),inplace=True)

In [0]:
X_validation['Months_Since_Deliquency'].fillna(X_validation['Months_Since_Deliquency'].mean(),inplace=True)

In [0]:
test_data['Months_Since_Deliquency'].fillna(test_data['Months_Since_Deliquency'].mean(),inplace=True)

In [386]:
X_train.isna().sum()

Loan_Amount_Requested      0
Length_Employed            0
Home_Owner                 0
Annual_Income              0
Income_Verified            0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Gender                     0
dtype: int64

In [387]:
X_validation.isna().sum()

Loan_Amount_Requested      0
Length_Employed            0
Home_Owner                 0
Annual_Income              0
Income_Verified            0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Gender                     0
dtype: int64

In [388]:
test_data.isna().sum()

Loan_Amount_Requested      0
Length_Employed            0
Home_Owner                 0
Annual_Income              0
Income_Verified            0
Purpose_Of_Loan            0
Debt_To_Income             0
Inquiries_Last_6Mo         0
Months_Since_Deliquency    0
Number_Open_Accounts       0
Total_Accounts             0
Gender                     0
dtype: int64

In [0]:
## Label Encoding
label_encode_experience = LabelEncoder()
label_encode_interest_rate = LabelEncoder()

In [0]:
X_train['Length_Employed'] = label_encode_experience.fit_transform(X_train['Length_Employed'])

In [0]:
X_validation['Length_Employed'] = label_encode_experience.transform(X_validation['Length_Employed'])

In [0]:
test_data['Length_Employed'] = label_encode_experience.transform(test_data['Length_Employed'])

In [0]:
y_train = label_encode_interest_rate.fit_transform(y_train)

In [0]:
y_validation = label_encode_interest_rate.fit_transform(y_validation)

In [0]:
## Get Dummies
X_train_dummy_cat = X_train[['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender']]

In [0]:
X_train_dummy_cat=pd.get_dummies(columns = ['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender'], 
                                 data = X_train_dummy_cat, 
                                 drop_first= True)

In [397]:
X_train_dummy_cat.head(3)

,Home_Owner_None,Home_Owner_Other,Home_Owner_Own,Home_Owner_Rent,Income_Verified_VERIFIED - income source,Income_Verified_not verified,Purpose_Of_Loan_credit_card,Purpose_Of_Loan_debt_consolidation,Purpose_Of_Loan_educational,Purpose_Of_Loan_home_improvement,Purpose_Of_Loan_house,Purpose_Of_Loan_major_purchase,Purpose_Of_Loan_medical,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Gender_Male
Loan_ID,,,,,,,,,,,,,,,,,,,,
10003909,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
10120777,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
10129626,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
X_validation_dummy_cat = X_validation[['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender']]
X_validation_dummy_cat=pd.get_dummies(columns = ['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender'], 
                                 data = X_validation_dummy_cat, 
                                 drop_first= True)

In [399]:
X_validation_dummy_cat.head(3)

,Home_Owner_None,Home_Owner_Other,Home_Owner_Own,Home_Owner_Rent,Income_Verified_VERIFIED - income source,Income_Verified_not verified,Purpose_Of_Loan_credit_card,Purpose_Of_Loan_debt_consolidation,Purpose_Of_Loan_educational,Purpose_Of_Loan_home_improvement,Purpose_Of_Loan_house,Purpose_Of_Loan_major_purchase,Purpose_Of_Loan_medical,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Gender_Male
Loan_ID,,,,,,,,,,,,,,,,,,,,
10086865,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
10123833,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
10066639,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [0]:
Test_dummy_cat = test_data[['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender']]
Test_dummy_cat = pd.get_dummies(columns = ['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender'], 
                                data = Test_dummy_cat, 
                                drop_first= True)

In [0]:
X_train.drop(['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender'],axis=1,inplace=True)

In [0]:
X_validation.drop(['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender'],axis=1,inplace=True)

In [0]:
test_data.drop(['Home_Owner','Income_Verified','Purpose_Of_Loan','Gender'],axis=1,inplace=True)

In [403]:
print(X_train_dummy_cat.shape)
print(X_validation_dummy_cat.shape)
print(Test_dummy_cat.shape)

(131447, 20)
(32862, 20)
(109541, 20)


In [407]:
print(X_train.shape)
print(X_validation.shape)
print(test_data.shape)

(131447, 8)
(32862, 8)
(109541, 8)


In [0]:
train1_data = pd.concat([X_train, X_train_dummy_cat], axis=1,sort=False)

In [0]:
validation_data = pd.concat([X_validation, X_validation_dummy_cat], axis=1,sort=False)

In [0]:
Test1_Data = pd.concat([test_data, Test_dummy_cat], axis=1,sort=False)


In [0]:
temp = pd.DataFrame(y_train)

In [412]:
temp.isna().sum()

0    0
dtype: int64

In [0]:
temp = Test1_Data.copy()

In [0]:
#xgb = XGBClassifier() ## Instantiate XGBClassifier model

#optimization_dict = {'max_depth': [2,3,4,5,6,7], ## trying with different max_depth,n_estimators to find best model
#                     'n_estimators': [50,60,70,80,90,100,150,200]} 

## Build best model with Grid Search params
#model = GridSearchCV(xgb, ## XGB model
#                     optimization_dict, ## dictory with different max_depth,n_estimators
#                     scoring='accuracy', ## on which parameter we are interested
#                     verbose=1, ## for messaging purpose
#                     n_jobs=-1) ## Number of jobs to run in parallel. ''-1' means use all processors

#%time model.fit(train_data, y_train) ## Fit a model
#print(model.best_score_) ## Display best score calues
#print(model.best_params_) ## Display best parameters

In [414]:
## The best params were found after grid search CV (above code)
model = XGBClassifier(max_depth=6, n_estimators=200,learning_rate = 0.001,
                      booster ='gbtree',random_state=1234)
%time model.fit(train1_data, y_train)

CPU times: user 2min 24s, sys: 208 ms, total: 2min 24s
Wall time: 2min 26s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.001, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=1234,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
## Get the predictions on train data
train_pred = model.predict(train1_data)

In [416]:
## Display accuracy value for train data
print("Train Accuracy :",accuracy_score(y_train,train_pred))

Train Accuracy : 0.5115065387570655


In [0]:
## Get the predictions on validation data
validation_pred = model.predict(validation_data)

In [418]:
## Display  accuracy value for validation data
print("Validation Accuracy :",accuracy_score(y_validation,validation_pred))

Validation Accuracy : 0.5079727344653399


In [419]:
## Get the confusion matrix for train data
confusion_matrix_train = confusion_matrix(y_train, train_pred)
print(confusion_matrix_train)

[[ 3137 19081  4864]
 [ 2060 39071 15417]
 [  566 22223 25028]]


In [420]:
## Get the confusion matrix for validation data
confusion_matrix_validation = confusion_matrix(y_validation, validation_pred)
print(confusion_matrix_validation)

[[ 739 4732 1253]
 [ 523 9607 3902]
 [ 134 5625 6347]]


In [0]:
## Get the predictions on test data
y_pred = model.predict(temp)

In [299]:
y_pred

array([1, 0, 2, ..., 1, 2, 1])

In [0]:
temp1 = temp.copy()

In [426]:
temp1.head()

,Loan_Amount_Requested,Length_Employed,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Home_Owner_None,Home_Owner_Other,Home_Owner_Own,Home_Owner_Rent,Income_Verified_VERIFIED - income source,Income_Verified_not verified,Purpose_Of_Loan_credit_card,Purpose_Of_Loan_debt_consolidation,Purpose_Of_Loan_educational,Purpose_Of_Loan_home_improvement,Purpose_Of_Loan_house,Purpose_Of_Loan_major_purchase,Purpose_Of_Loan_medical,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Gender_Male
Loan_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10164310,27500.0,2,129000.0,12.87,0,68.000000,10,37,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
10164311,26000.0,2,110000.0,11.37,0,33.914684,6,23,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
10164312,6075.0,0,75000.0,6.83,2,33.914684,5,20,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
10164313,12000.0,2,73000.0,7.76,0,33.914684,6,8,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
10164314,35000.0,0,156000.0,9.62,0,26.000000,9,21,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
temp1['Interest_Rate'] = label_encode_interest_rate.inverse_transform(y_pred)

In [0]:
temp1.reset_index(inplace=True)


In [306]:
Test_Data

,Loan_Amount_Requested,Length_Employed,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts
0,7000.0,2,129000.0,12.87,0,68.000000,10,37
1,30000.0,2,110000.0,11.37,0,33.914684,6,23
2,24725.0,0,75000.0,6.83,2,33.914684,5,20
3,16000.0,2,73000.0,7.76,0,33.914684,6,8
4,17000.0,0,156000.0,9.62,0,26.000000,9,21
...,...,...,...,...,...,...,...,...
109536,8000.0,10,120000.0,8.15,2,32.000000,13,57
109537,21200.0,2,125000.0,9.08,1,17.000000,9,34
109538,16000.0,6,37850.0,11.60,0,33.914684,10,11
109539,14000.0,7,30000.0,24.88,0,33.914684,5,6


In [0]:
to_submit_1 = temp1[['Loan_ID', 'Interest_Rate']]

In [430]:
to_submit_1.shape

(109541, 2)

In [433]:
Test1_Data.shape


(109541, 28)

In [434]:
to_submit_1.Interest_Rate.value_counts()


2    71330
3    36342
1     1869
Name: Interest_Rate, dtype: int64

In [439]:
!ls

gdrive	sample_data  XGBoost.csv


In [0]:
## store to_submit into csv file with name xgb_model 
to_submit_1.to_csv('/contents/gdrive/My Drive/Analytic_vidya/XGB.csv',index = False)

In [0]:
### Gradient Boosting
model = GradientBoostingClassifier(n_estimators=200,learning_rate=0.01)

In [444]:
%time model.fit(train1_data, y_train)

CPU times: user 2min 54s, sys: 713 ms, total: 2min 55s
Wall time: 2min 58s


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [446]:
## Display accuracy value for train data
print("Train Accuracy :",accuracy_score(y_train,train_pred))

Train Accuracy : 0.5115065387570655


In [449]:
## Display  accuracy value for validation data
print("Validation Accuracy :",accuracy_score(y_validation,validation_pred))

Validation Accuracy : 0.5079727344653399


In [450]:
## Get the confusion matrix for train data
confusion_matrix_train = confusion_matrix(y_train, train_pred)
print(confusion_matrix_train)

[[ 3137 19081  4864]
 [ 2060 39071 15417]
 [  566 22223 25028]]


In [454]:
## Get the confusion matrix for validation data
confusion_matrix_validation = confusion_matrix(y_validation, validation_pred)
print(confusion_matrix_validation)

[[ 739 4732 1253]
 [ 523 9607 3902]
 [ 134 5625 6347]]


In [0]:
## Get the predictions on test data
y_pred = model.predict(temp)

In [0]:
temp1 = temp.copy()

In [0]:
temp1['Interest_Rate'] = label_encode_interest_rate.inverse_transform(y_pred)

In [0]:
temp1.reset_index(inplace=True)


In [0]:
to_submit_2 = temp1[['Loan_ID', 'Interest_Rate']]


In [462]:
to_submit_2.shape

(109541, 2)

In [463]:
test_data.shape

(109541, 8)

In [464]:
to_submit_2.Interest_Rate.value_counts()

2    73869
3    35628
1       44
Name: Interest_Rate, dtype: int64

In [467]:
!ls

GDB1.csv  gdrive  sample_data  XGBoost.csv


In [0]:
os.chdir('//gdrive/My Drive/Analytic_vidya')

In [0]:
## store to_submit into csv file with name xgb_model 
to_submit_2.to_csv('/content/gdrive/My Drive/Analytic_vidya/GDB1.csv',index = False)